# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [4]:
!unzip News_Category_Dataset_v2.zip

Archive:  News_Category_Dataset_v2.zip
  inflating: News_Category_Dataset_v2.json  


In [8]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [9]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [10]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [12]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=2)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [14]:
Word2VecKeyedVectors = model.train(data, total_examples=model.corpus_count, epochs=10)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [16]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [24]:
wv.most_similar('Texas')

[('Oklahoma', 0.801356315612793),
 ('Maryland', 0.784896731376648),
 ('Ohio', 0.7804616689682007),
 ('Oregon', 0.7787513136863708),
 ('Florida', 0.7644636034965515),
 ('Louisiana', 0.7631367444992065),
 ('California', 0.7619392275810242),
 ('Arkansas', 0.7561051249504089),
 ('Pennsylvania', 0.7544770836830139),
 ('Utah', 0.7518435716629028)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [19]:
wv.most_similar(negative='Texas')

[('once-reliable', 0.42561930418014526),
 ('-shaped', 0.40178290009498596),
 ('Recognize', 0.3873647153377533),
 ('maitre', 0.38679778575897217),
 ('Parent/Grandparent', 0.37388160824775696),
 ('Animal-rescue', 0.37099024653434753),
 ('much-vaunted', 0.3706092834472656),
 ('Warlike', 0.3594048023223877),
 ('Likened', 0.3522931933403015),
 ('Memorialize', 0.3508149981498718)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [20]:
wv['Texas']

array([ 0.96031797,  2.5434957 , -1.2570865 ,  0.89356387, -2.790635  ,
       -0.14214705, -0.70424235, -0.76424813,  1.1327114 ,  1.2764654 ,
       -1.0643024 ,  0.01645868, -0.87235135, -0.72728   , -1.499232  ,
       -1.1980426 , -3.5688176 ,  0.80371904, -1.2757051 , -0.0336359 ,
        0.77995706,  0.5990114 , -0.6062373 , -1.5494844 , -0.5431635 ,
        0.8627928 , -1.5520065 ,  0.35552213,  0.88632447, -1.5926597 ,
       -1.3981376 , -1.3342268 ,  1.5357598 , -1.1918459 , -0.9830081 ,
        0.15766582,  0.29995522,  3.0346398 ,  0.422598  ,  0.454599  ,
       -1.414877  , -0.54591346, -3.0276146 ,  3.640286  ,  0.6715398 ,
        1.6645384 , -1.2877609 , -0.33818832,  1.459933  ,  0.1018566 ,
       -1.4043655 ,  1.29825   , -1.7493732 ,  1.0596164 , -0.81616825,
        1.7604493 , -1.6529588 ,  1.5375378 ,  2.0422919 , -4.1054306 ,
       -0.3223317 ,  1.462561  ,  1.0165282 , -1.0590544 , -0.5622955 ,
       -0.83041865,  2.1492522 ,  0.2284799 , -0.5196874 ,  0.96

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [25]:
wv.vectors

array([[-1.3159347e+00, -1.0054803e+00,  6.3733536e-01, ...,
         1.5532901e+00,  2.6385407e+00,  9.8457009e-01],
       [-2.3606229e+00, -1.4180449e+00,  2.2406167e-01, ...,
         8.1344074e-01,  1.4994118e+00, -1.9626223e-02],
       [-5.9571940e-01, -2.3897314e-01,  2.2719083e+00, ...,
         1.9176471e+00,  1.1175770e+00, -5.1308360e-02],
       ...,
       [-5.5508982e-02,  5.1394932e-02,  3.1176660e-02, ...,
         6.9920532e-02, -1.8361060e-01, -4.0869903e-02],
       [-1.0501448e-01, -1.2940507e-02,  1.8487325e-01, ...,
         1.3165738e-01, -3.0934706e-02,  1.6105356e-02],
       [ 4.1808665e-02, -5.7883416e-03,  4.3601416e-02, ...,
         3.3238836e-02, -3.4866933e-02, -1.4091196e-03]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [27]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('ssy', 0.44483381509780884),
 ('Katwe', 0.4429285526275635),
 ('glug', 0.4339343309402466),
 ('reminiscent', 0.43198928236961365),
 ('princess', 0.4312226176261902),
 ("'Foulard", 0.4298449754714966),
 ('title', 0.4288977384567261),
 ('embodiment', 0.4154399037361145),
 ('dames', 0.41451725363731384),
 ('Miniaturization', 0.41206979751586914)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!